In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import pprint
from __future__ import division
from __future__ import print_function
from datetime import datetime
import calendar
from zipfile import ZipFile
import os
import sys
import dateutil
import zipfile
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'])

In [11]:
# lets get the latest packetloss and latency data
# it seems that elastic search is not really close to realtime, but rather a few hours behind?
# Or maybe the timezone for the timestamps is off?

CERN_data = {'name': "CERN", "latency": "128.142.223.247", "throughput": "128.142.223.246"}
TRIUMF_data = {'name': "TRIUMF", "latency": "206.12.9.2", "throughput": "206.12.9.1"}
src_data = CERN_data
dest_data = TRIUMF_data
data = pd.DataFrame()
response_size = 10
es_sorting = "timestamp:desc"
es_query_forth = \
{
    "query": {
        "filtered": {
            "query": {
                "match_all": {}
            },
            "filter": {
                "bool": {
                    "must":[
                        { 'term': { 'src': src_data["latency"] } },
                        { 'term': { 'dest': dest_data["latency"] } },
                    ]
                }
            }
        }
    }
}
es_query_back = \
{
    "query": {
        "filtered": {
            "query": {
                "match_all": {}
            },
            "filter": {
                "bool": {
                    "must":[
                        { 'term': { 'src': src_data["latency"] } },
                        { 'term': { 'dest': dest_data["latency"] } },
                    ]
                }
            }
        }
    }
}

es_index = "network_weather_2-*"

# Collect all needed data raw data
event_type = 'latency'
response_latency_forth = es.search(index=es_index, body=es_query_forth, size=response_size, sort=es_sorting, doc_type = event_type, request_timeout=10)
response_latency_back = es.search(index=es_index, body=es_query_back, size=response_size, sort=es_sorting, doc_type = event_type, request_timeout=10)
event_type = 'packet_loss_rate'
response_packetloss = es.search(index=es_index, body=es_query_forth, size=response_size, sort=es_sorting, doc_type = event_type, request_timeout=10)

# analyze data
for dataPoint in response_latency_forth['hits']['hits']:
    time = dateutil.parser.parse(dataPoint['_source']['timestamp'])
    timestamp_epoch = calendar.timegm(time.timetuple())
    data.set_value(timestamp_epoch, 'delay_mean_forth', dataPoint['_source']['delay_mean'])
    
for dataPoint in response_latency_back['hits']['hits']:
    time = dateutil.parser.parse(dataPoint['_source']['timestamp'])
    timestamp_epoch = calendar.timegm(time.timetuple())
    data.set_value(timestamp_epoch, 'delay_mean_back', dataPoint['_source']['delay_mean'])
    
for dataPoint in response_packetloss['hits']['hits']:
    time = dateutil.parser.parse(dataPoint['_source']['timestamp'])
    timestamp_epoch = calendar.timegm(time.timetuple())
    data.set_value(timestamp_epoch, 'packet_loss', dataPoint['_source']['packet_loss'])



In [3]:
data

,delay_mean_forth,delay_mean_back,packet_loss
1473174900,76.425722,76.425722,0.0
1473174600,76.668233,76.668233,0.0
1473174300,76.636967,76.636967,0.0
1473174000,76.596900,76.596900,0.0
1473173700,76.537067,76.537067,0.0
1473173400,76.427667,76.427667,0.0
1473173100,76.418000,76.418000,0.0
1473172800,76.722333,76.722333,0.0
1473172500,76.364400,76.364400,0.0
1473172200,76.398200,76.398200,0.0


In [ ]:
# this method should only be used for "small" timeframes containig less than 10.000 results
# otherwise it will crash
# a good cache size could be 1000
def collect_and_cleanRawData_RealTime(es, src_site, dest_site, timeframe, cache_size):
    print("--- Starting data collection for {} to {}".format(src_site, dest_site))
    es_sorting = "timestamp:desc"
    es_size = cache_size
    es_query = \
    {
        "query": {
            "filtered": {
                "query": {
                    "match_all": {}
                },
                "filter": {
                    "bool": {
                        "must":[
                            { 'term': { 'srcSite': src_site } },
                            { 'term': { 'destSite': dest_site } },
                            { 'range': { 'timestamp': timeframe } }
                        ]
                    }
                }
            }
        }
    }
    es_index = "network_weather_2-*"
    
    # Collect latency raw data
    print("--- Collectiong latency data")
    event_type = 'latency'
    es_from = 0
    response = es.search(index=es_index, body=es_query, size=10, from_=es_from, sort=es_sorting, doc_type = event_type, request_timeout=600)
    numDataPoints = response['hits']['total']
    while (es_from <= numDataPoints):
        response = es.search(index=es_index, body=es_query, size=es_size, from_=es_from, sort=es_sorting, doc_type = event_type, request_timeout=600)
        for dataPoint in response['hits']['hits']:
            timestamp_epoch = dataPoint['_source']['timestamp']
            put_data(src_site, dest_site, timestamp_epoch, 'iso_8601', dataPoint['_source']['timestamp'])
            put_data(src_site, dest_site, timestamp_epoch, 'delay_median', dataPoint['_source']['delay_median'])
            put_data(src_site, dest_site, timestamp_epoch, 'delay_mean', dataPoint['_source']['delay_mean'])
            put_data(src_site, dest_site, timestamp_epoch, 'delay_sd', dataPoint['_source']['delay_sd'])
        es_from = es_from + es_size
    print('Number of raw records of {} is {}'.format(event_type, numDataPoints))
        
    # Collect packet_loss_rate raw data
    print("--- Collectiong packet_loss_rate data")
    event_type = 'packet_loss_rate'
    es_from = 0
    response = es.search(index=es_index, body=es_query, size=10, from_=es_from, sort=es_sorting, doc_type = event_type, request_timeout=600)
    numDataPoints = response['hits']['total']
    while (es_from <= numDataPoints):
        response = es.search(index=es_index, body=es_query, size=es_size, from_=es_from, sort=es_sorting, doc_type = event_type, request_timeout=600)
        for dataPoint in response['hits']['hits']:
            timestamp_epoch = dataPoint['_source']['timestamp']
            put_data(src_site, dest_site, timestamp_epoch, 'iso_8601', dataPoint['_source']['timestamp'])
            put_data(src_site, dest_site, timestamp_epoch, 'packet_loss', dataPoint['_source']['packet_loss'])
        es_from = es_from + es_size
    print('Number of raw records of {} is {}'.format(event_type, numDataPoints))
    
    # Collect throughput raw data
    print("--- Collectiong throughput data")
    event_type = 'throughput'
    es_from = 0
    response = es.search(index=es_index, body=es_query, size=10, from_=es_from, sort=es_sorting, doc_type = event_type, request_timeout=600)
    numDataPoints = response['hits']['total']
    while (es_from <= numDataPoints):
        response = es.search(index=es_index, body=es_query, size=es_size, from_=es_from, sort=es_sorting, doc_type = event_type, request_timeout=600)
        for dataPoint in response['hits']['hits']:
            timestamp_epoch = dataPoint['_source']['timestamp']
            put_data(src_site, dest_site, timestamp_epoch, 'iso_8601', dataPoint['_source']['timestamp'])
            put_data(src_site, dest_site, timestamp_epoch, 'throughput', dataPoint['_source']['throughput'])
        es_from = es_from + es_size
    
    print('Number of raw records of {} is {}'.format(event_type, numDataPoints))
    # View statistics of raw_data_pool (already de-duplicated)
    print("")
    print('De-duplication result:')
    print(raw_data_pool[src_site][dest_site].count(axis='index'))